In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SRBC']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('sheep red blood cells', 278), ('stroma rich bladder cancer model', 3)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'sheep red blood cells': 'sheep_red_blood_cells',
  'stroma rich bladder cancer model': 'ungrounded'},
 {'sheep_red_blood_cells': 'sheep_red_blood_cells'},
 []]

In [11]:
grounding_map, names, pos_labels = [{'sheep red blood cells': 'sheep_red_blood_cells',
  'stroma rich bladder cancer model': 'ungrounded'},
 {'sheep_red_blood_cells': 'sheep_red_blood_cells'},
 []]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {'HGNC:1639': 'CD2'}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[('HGNC:1639', 'HGNC:1639', 106)]

In [20]:
intersection2

[('sheep_red_blood_cells', 'HGNC:1639', 0), ('ungrounded', 'HGNC:1639', 0)]

In [21]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['CD2', 'sheep red blood cell receptor'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [22]:
names.update(additional_entities)
pos_labels.extend(additional_entities)

In [23]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-17 04:10:49] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-17 04:10:55] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9711908813440294 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [24]:
classifier.stats

{'label_distribution': {'sheep_red_blood_cells': 220,
  'ungrounded': 1,
  'HGNC:1639': 106},
 'f1': {'mean': 0.971191, 'std': 0.028329},
 'precision': {'mean': 0.980476, 'std': 0.023924},
 'recall': {'mean': 0.962338, 'std': 0.035529},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:1639': {'f1': {'mean': 0.971191, 'std': 0.028329},
  'pr': {'mean': 0.962338, 'std': 0.035529},
  'rc': {'mean': 0.980476, 'std': 0.023924}},
 'sheep_red_blood_cells': {'f1': {'mean': 0.984219, 'std': 0.011489},
  'pr': {'mean': 0.990909, 'std': 0.011134},
  'rc': {'mean': 0.977677, 'std': 0.014056}}}

In [25]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [26]:
disamb.dump(model_name, results_path)

In [27]:
print(disamb.info())

Disambiguation model for SRBC

Produces the disambiguations:
	CD2*	HGNC:1639
	sheep_red_blood_cells	sheep_red_blood_cells

Class level metrics:
--------------------
Grounding            	Count	F1     
sheep_red_blood_cells	220	0.98422
                  CD2*	106	0.97119
           Ungrounded	  1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.97119
	Precision:	0.98048
	Recall:		0.96234

* Positive labels
See Docstring for explanation



In [29]:
model_to_s3(disamb)

In [29]:
texts = [text for text, pred in zip(all_texts.values(), disamb.disambiguate(all_texts.values())) if pred[0] == 'HGNC:1639']

In [32]:
'\n\n'.join(texts)

'Passive hemagglutination test (HP) after binding of ABS antigen to SRBC by chromium chloride, lymphocyte transformation (TTL) and inhibition of leukocyte migration (TML) tests were compared to tube agglutination and complement fixation tests, and to clinical symptoms for diagnosis of brucellosis. Both TTL and TML persist for years after contact with brucella. HP is the most reliable serologic test.\n\n\nResting natural killer (NK) cells express the p75 chain of the IL-2 receptor (IL-2R beta) and most NK cells express the CD2 (erythrocyte rosette) receptor. The cell adhesion molecule, LFA-3, is a natural co-ligand for CD2. Tac antigen (IL-2R alpha), a p55 IL-2R subunit, can be expressed after NK activation and may play a role in IL-2-induced NK proliferation. Little is known of the molecular mechanisms underlying cytokine production in NK cells. We investigated the roles of IL-2R alpha, IL-2R beta, and CD2/LFA-3 in the molecular regulation of NK cell granulocyte/macrophage-colony-stimu